In [1]:
import os
import sys

In [2]:
sys.path.append('../lib')

In [3]:
import cov_independence_test
import hsic_independence_test
import gen_processes

In [4]:
from importlib import reload

In [5]:
reload(cov_independence_test);
reload(hsic_independence_test);
reload(gen_processes);

In [9]:
sys.path.append('../lib/sigkernel')

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.tsa.ar_model as ar
import torch
torch.set_default_dtype(torch.double)
import sigkernel_scale

In [11]:
import pprint
pp = pprint.PrettyPrinter(indent = 4)

In [12]:
import matplotlib.pyplot as plt
import pickle

In [70]:
rng = np.random.default_rng(1234)

In [115]:
a = torch.tensor([i for i in range(10)])
b = torch.tensor([i for i in range(10,15)])
b_same = torch.tensor([i for i in range(10,20)])

In [95]:
len_vct_a = 30
len_vct_b = 20

In [122]:
a_tiled = torch.tile(a, (len_vct_a, 1))
b_tiled = torch.tile(b, (len_vct_b, 1))
b_tiled_same = torch.tile(b_same, (len_vct_b, 1))

In [121]:
a, b

(tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), tensor([10, 11, 12, 13, 14]))

In [97]:
print(a_tiled.shape) # time x batch
print(b_tiled.shape) # time x batch

torch.Size([30, 10])
torch.Size([20, 5])


In [99]:
scale = torch.einsum('ij,kl->jlik',a_tiled,b_tiled)

In [111]:
scale[2, 2, :, :];

In [123]:
scale = torch.einsum('ji,ki->ijk',a_tiled,b_tiled_same)

In [128]:
scale[3, :, :]

tensor([[39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39,
         39, 39],
        [39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39,
         39, 39],
        [39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39,
         39, 39],
        [39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39,
         39, 39],
        [39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39,
         39, 39],
        [39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39,
         39, 39],
        [39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39,
         39, 39],
        [39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39,
         39, 39],
        [39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39,
         39, 39],
        [39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39,
         39, 39],
        [3

In [13]:
with open('pickles/comparison_ds_for_normalized_sig_pde_kernel.pickle', 'rb') as inf:
    INPUT = pickle.load(inf)

In [14]:
paths_ll = torch.tensor(INPUT['processes_lead_lag'])

In [15]:
def psi(x, M=4, a=1):
    x = x ** 2

    if x <= M:
        return x

    return M + M ** (1 + a) * (M ** (-a) - x ** (-a)) / a

In [16]:
print(INPUT['gram_matrix'][0, 0])

12.380862586180728


In [29]:
print(INPUT['comment'])
print(paths_ll.shape)

Process indices: Sample, Time, Lead/Lag
torch.Size([20, 30, 2])


In [20]:
sig_kernel = sigkernel_scale.SigKernel(sigkernel_scale.LinearKernel(), 3)

In [31]:
kernel_xx = sig_kernel.compute_kernel(paths_ll[0:1, :, :], paths_ll[0:1, :, :])

In [34]:
target = psi(np.sqrt(kernel_xx.item()))

In [35]:
target

6.9782870722007795

In [41]:
f = lambda lam: target - sig_kernel.compute_kernel(lam*paths_ll[0:1, :, :], lam*paths_ll[0:1, :, :]).item()

In [22]:
from scipy.optimize import brentq

In [43]:
brentq(f, 0, 1.)

0.8627978998614458

In [23]:
scales = torch.zeros(20)

In [24]:
for i in range(20):
    kernel_xx = sig_kernel.compute_kernel(paths_ll[i:i+1, :, :], paths_ll[i:i+1, :, :])
    target = psi(np.sqrt(kernel_xx.item()))
    f = lambda lam: target - sig_kernel.compute_kernel(paths_ll[i:i+1, :, :], lam*paths_ll[i:i+1, :, :]).item()
    print(brentq(f, 0., 1.1, xtol=1e-20))
    scales[i] = brentq(f, 0., 1.1, xtol=1e-20)

0.8630038091506689
0.9998016300589121
0.9999999999999536
0.8940687347622606
0.9559379933582218
0.9865845484043637
0.9736827828678053
0.980207174382617
0.9921669659308869
0.9216126190278953
1.0000000000000253
0.7747179212101265
0.9905295138762319
0.8615076050562425
0.9542041509589576
0.9369761889352346
0.9999837935174736
1.000000000000138
0.8919309366176795
0.8931298905858582


In [26]:
scales_new = torch.zeros(20)

In [35]:
sig_kernel = sigkernel_scale.SigKernel(sigkernel_scale.RBFKernel(0.3), 3)

In [36]:
for i in range(20):
    kernel_xx = sig_kernel.compute_kernel(paths_ll[i:i+1, :, :], paths_ll[i:i+1, :, :])
    target = psi(np.sqrt(kernel_xx.item()))
    f = lambda lam: target - sig_kernel.compute_kernel(paths_ll[i:i+1, :, :], paths_ll[i:i+1, :, :], torch.tensor([lam]), torch.tensor([lam])).item()
    print(brentq(f, 0., 1.1, xtol=1e-20))
    scales_new[i] = brentq(f, 0., 1.1, xtol=1e-20)

0.664972219150016
0.7374572449614519
0.66951012759249
0.6136075369759413
0.678762357766056
0.6814555752116332
0.6406308266176514
0.6927950237940864
0.6744137173084176
0.6699180026650583
0.7014539200743989
0.6515355115733266
0.6694667508059865
0.6800968649397722
0.7013901860468926
0.6610658298741133
0.6526914858288777
0.7034896148662164
0.6302312963942747
0.6465006099885344


In [30]:
sig_kernel.compute_kernel(paths_ll[0:1, :, :], paths_ll[0:1, :, :], torch.tensor([0.863]), torch.tensor([0.863]))

tensor([6.9750])

In [49]:
kernel_vs_time = [sig_kernel.compute_kernel(paths_ll[0:1, :i+1, :], paths_ll[0:1, :i+1, :]) for i in range(1, 30)]

In [50]:
normalized_kernel_vs_time = [psi(torch.sqrt(k)) for k in kernel_vs_time]

In [71]:
def get_tv_of_path(path): #correct path length estimate
    lin_ker_values = path @ path.T
    vct_of_sq_distances = torch.diagonal(lin_ker_values)[:-1] + torch.diagonal(lin_ker_values)[1:] - 2*torch.diagonal(lin_ker_values, offset = 1)
    return torch.sqrt(vct_of_sq_distances).sum()

In [72]:
tv_path = get_tv_of_path(paths_ll[0, :, :]).item()

In [73]:
tv_path

9.9237479739895

Note: kernel_vs_time agrees with simple inner product of signatures in esig package

In [54]:
kernel_vs_time[-1]

tensor([15.6113])

In [55]:
kernel_last_ts = normalized_kernel_vs_time[-1].item()

In [56]:
kernel_last_ts

6.975099157668957

In [26]:
import scipy as scp
from scipy.optimize import brentq
from scipy.special import iv

In [86]:
f = lambda tv: 12.38 - iv(0, 0.96287837*tv*2)

In [87]:
brentq(f,  0, 20)

2.131768334121332

In [59]:
get_tv_of_path(paths_ll[0, :, :])

tensor(9.9237)

In [114]:
(paths_ll[0, -1, :] - paths_ll[0, 0, :]).numpy()

array([-0.97565892, -0.96089106])